In [4]:
import numpy as np
import pandas as pd 

In [7]:
#Sample Dataset building

def build_dataset(size):
  size_pos = 64
  size_neg = 64
  size_noise = 64

  pos_cols = ["pos_sample_%s" % i for i in range(size_pos)]
  neg_cols = ["neg_sample_%s" % i for i in range(size_neg)]
  noise_cols = ["noise_%s" % i for i in range(size_noise)]

  pos = {i : np.random.uniform(-1, 1, size = size) for i in pos_cols}
  neg = {i : np.random.uniform(-1, 1, size = size) for i in neg_cols}
  noise = {i : np.random.uniform(-1, 1, size = size) for i in noise_cols}

  df = pd.DataFrame({**pos, **neg, **noise})

  df["target"] = df.apply(lambda x : sum(
            [x[k] * (i+1)/size_pos for i, k in enumerate(pos_cols)] + 
            [-x[k] * (i+1)/size_neg for i, k in enumerate(neg_cols)], axis = 1)
  )

  coefs = [(i+1)/size_pos for i in range(size_pos)] + [-(i+1)/size_neg for i in range(size_neg)] + [0 for i in range(size_noise)]

  return np.array(df[pos_cols + neg_cols + noise_cols]), np.array(df["target"]), coefs

In [ ]:
cols, target, coefs = build_dataset(128)

In [ ]:
def integrated_grads(batch, batch_base, model): #model is a regressor to fit x,y
  mean_grad = 0 #integral is calculated by y_val of gradient * x (1/n)
  n = 100

  for i in range(1, n+1):
    x = batch_base + (batch - batch_base) * (i/n)
    x.requires_grad = True
    y_val = model(x)
    (grad, _) = torch.autograd.grad(y_val, x)
    mean_grad += grad * (1/n)

  integrated_gradients = (batch - batch_base) * mean_grad

  return integrated_gradients, mean_grad  